# Lectura de archivos 


Semana Tec

Profesores: Luz Eunice Angeles /José Ignacio Treviño

Qué aprenderemos: 

- Subir archivos desde el sistema de archivos local 
- Descargar archivos en el slistema de archivos local 
- Acceder a google drive 
  - Activarlo de en el entorno de ejecución
  - Usar una API : PyDrive
- Crear y leer datos de google sheets


## Subir archivos desde el sistema de archivos local

<code>files.upload</code> es usado para cargar los archivos y conocer el diccionario de los archivos que se cargaron.
El diccionario está codificado según el nombre de archivo y los valores son los datos que se subieron.

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Auto.csv to Auto.csv
User uploaded file "Auto.csv" with length 17681 bytes


## Descargar archivos en el sistema de archivos local

<code>files.download</code> invocará una descarga del archivo desde el navegador a tu computadora local.


In [5]:
from google.colab import files

with open('exampleSemanaTec.txt', 'w') as f:
  f.write('some content')

files.download('exampleSemanaTec.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
ls

Auto.csv  exampleSemanaTec.txt  sample_data/


# Google Drive

Puedes acceder a archivos en Drive de varias maneras, incluidas las siguientes:
- Activación de Google Drive en la máquina virtual del entorno de ejecución
- Usar un wrapper alrededor de la API, como <a href="https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html">PyDrive</a>
- Usar la <a href="https://developers.google.com/drive/v3/web/about-sdk">API nativa de REST</a>



Puedes ver ejemplos de cada caso más abajo.

## Activar Google Drive de forma local

El ejemplo siguiente muestra cómo activar Google Drive en el entorno de ejecución mediante un código de autorización y cómo escribir y leer archivos allí. Una vez ejecutado, podrás ver el archivo nuevo &#40;<code>foo.txt</code>&#41; en <a href="https://drive.google.com/">https://drive.google.com/</a>.

Ten en cuenta que solo se puede leer, escribir o mover archivos. 

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
with open('/content/drive/My Drive/SemanaTec/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/semanaTec/foo.txt

cat: '/content/drive/My Drive/semanaTec/foo.txt': No such file or directory


## PyDrive

En los ejemplos que aparecen a continuación, se muestran la autenticación y la carga y descarga de archivos mediante PyDrive. Puedes encontrar más ejemplos en la <a href="https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html">documentación de PyDrive</a>.

In [10]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Autentica y crea el cliente de PyDrive.


In [11]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Lista de ids de cada carpeta que tenemos en el google drive

In [12]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:       
    print ('title: %s, id: %s' % (file1['title'], file1['id']))


title: 2.1 - Básicos de Unix y GIT.ipynb, id: 1tQQ10aOzDTBRHFpGwtGwbRQndCIQ9UzO
title: SemanaTec, id: 1--HMaQYcJ3ZSuMKmgFg5ch78KbqwDamb
title: credenciales.json, id: 1dzEw4drci_dAZ7s-7dC0zI1a2P1RFUN8
title: apple-touch-icon-144x144-precomposed.png, id: 17dYj6rOjo1FlUhkuV9tjfEI5Uthwit0D
title: 2.3 - Actualizar repositorio.ipynb, id: 1-wwE8zCyuZiC2YN3AsjKV4PCQuvoVD8m
title: video1391604567.mp4, id: 1JV_pXPb1TZQvxLDC943HQRF2VHRtndNN
title: parte3.mp4, id: 1-Fp1ghWRSNCiTELZfOfW3pB2T6KgC7VP
title: Mapa sin nombre, id: 1UcgAhPFkMlCuO9LLMNYeWEYUnmoFU3Gd


In [21]:
fid= '1--HMaQYcJ3ZSuMKmgFg5ch78KbqwDamb'

Crea y sube un archivo de texto.


In [22]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt','parents':[{"id":fid}]})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1gKmjPwvznJUk-5pzgs1OlNkarRlhHSzl


Carga un archivo por su ID y luego imprime su contenido.


In [23]:
uploaded.get('id')

'1gKmjPwvznJUk-5pzgs1OlNkarRlhHSzl'

In [24]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


# Hojas de cálculo de Google

Los ejemplos de abajo usan la biblioteca de código abierto <a href="https://github.com/burnash/gspread"><code>gspread</code></a> para interactuar con Hojas de cálculo de Google.

Primero, instala el paquete usando <code>pip</code>.

In [16]:
!pip install --upgrade --quiet gspread

Importa la biblioteca, autentica y crea la interfaz para Hojas de cálculo.

In [17]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

A continuación, verás un pequeño conjunto de ejemplos de <code>gspread</code>. Puedes encontrar más ejemplos en la <a href="https://github.com/burnash/gspread#more-examples">página de GitHub de <code>gspread</code></a>.

## Crear una hoja nueva con datos de Python

In [18]:
sh = gc.create('My cool spreadsheet')

Después de ejecutar la celda anterior, se mostrará una nueva hoja de cálculo con el nombre "My cool spreadsheet" en <a href="https://sheets.google.com/">https://sheets.google.com</a>.

Abre la hoja nueva y agrega datos aleatorios.

In [19]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1a6f5LCHBim-aIpagTi6mRCEbgDnLCGa-aPn2iYfsJN8',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': "'Hoja 1'!A1:C2",
 'updatedRows': 2}

## Descargar datos de una hoja de cálculo en Python como un DataFrame de Pandas

Vuelve a leer los datos aleatorios que insertamos anteriormente y convierte el resultado en un <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">DataFrame de Pandas</a>.

In [20]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print('******************Lista de datos*********************')
print(rows)

import pandas as pd
dataset=pd.DataFrame.from_records(rows)
print('******************Data Frame*********************')
print(dataset)

******************Lista de datos*********************
[['10', '1', '3'], ['1', '3', '5']]
******************Data Frame*********************
    0  1  2
0  10  1  3
1   1  3  5
